<a href="https://colab.research.google.com/github/Kelnito/legendary-barnacle/blob/main/Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [2]:
origin = pandas.read_table("movies.csv", sep=",")

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
origin.tail()

,MovieID,Title,Genre
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


In [4]:
origin.isnull().sum().sum()

0

In [5]:
origin["Genre"] = origin["Genre"].apply(lambda i : " ".join(i.split("|")))

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


in Our Goal Using `TfidfVectorizer`, Replacing `Sci-Fi` to `SciFi`, `Film-Noir` to `Filnoir`

Others, Resulting `Sci-Fi` to (`Sci` + `Fi`), `Film-Noir` to (`Film` + `Noir`)

In [6]:
replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["Genre"] = origin["Genre"].str.replace(i, t)

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
nolist = "(no genres listed)"

origin = origin[origin["Genre"] != nolist]

origin.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
Genres = []

for insect in origin["Genre"].values:

  insect = insect.split(" ")

  for lst in insect:

    if lst not in Genres: Genres.append(lst)

len(Genres)

19

Overall, Movie Lens Small Datasets Have 19 Unique Genres

In [9]:
Genres[:5]

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

Converting Genres Data to TF-IDF (Term Frequency-Inverse Document Frequency) Matrix

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["Genre"])

metrics.shape

(9708, 19)

In [11]:
tfidf.get_feature_names_out().tolist()[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [12]:
len(tfidf.get_feature_names_out().tolist())

19

In [13]:
assert len(tfidf.get_feature_names_out().tolist()) == len(Genres), "Fail !"

In [14]:
len(metrics.todense())

9708

In [15]:
metrics.todense()[0]

matrix([[0.        , 0.4168036 , 0.51628768, 0.50489573, 0.26738778,
         0.        , 0.        , 0.        , 0.48301747, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [16]:
metrics.todense()[1]

matrix([[0.        , 0.51229321, 0.        , 0.62056723, 0.        ,
         0.        , 0.        , 0.        , 0.59367667, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [17]:
metrics.todense()[2]

matrix([[0.        , 0.        , 0.        , 0.        , 0.57070525,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.82115499,
         0.        , 0.        , 0.        , 0.        ]])

Perform Cosine Similarity to Measure Genres Similarity One Movie to Another

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(metrics)

similarities.shape

(9708, 9708)

In [19]:
len(similarities[0])

9708

In [20]:
similarities[0][:10]

array([1.        , 0.8136036 , 0.15259961, 0.13500041, 0.26738778,
       0.        , 0.15259961, 0.65470981, 0.        , 0.26241348])

Our Movies, Jumanji, Have 81 % Similarity on Genres to Toy Story

In [21]:
similarities[1][:10]

array([0.8136036 , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.80470368, 0.        , 0.32253235])

In [22]:
intler = lambda i : origin.iloc[i, 1]

titint = lambda t : origin[origin["Title"] == t].index.values[0]

In [23]:
t = titint("Jumanji (1995)")

t

1

In [24]:
moret = enumerate(similarities[int(t)])

molist = list(moret)

len(molist)

9708

In [25]:
molist[:5]

[(0, 0.8136035975025231), (1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0)]

In [26]:
reverse = True

molist = sorted(molist, key=lambda i:i[1], reverse=reverse)

molist[:5]

[(1, 1.0), (53, 1.0), (109, 1.0), (767, 1.0), (1514, 1.0)]

In [27]:
molist = molist[:5]

origin[origin.index.isin([i[0] for i in molist])]

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy


In [28]:
limit = 10

title = "Jumanji (1995)"

def output(title, limit):

  morate = enumerate(similarities[int(titint(title))])
  molist = list(morate)

  morter = sorted(molist, key=lambda i:i[1], reverse=reverse)
  molist = morter[:limit]

  molter = filter(lambda i : i[0] != titint(title), molist)
  molist = list(molter)

  titles = [intler(molist[i][0]) for i in range(len(molist))]
  result = origin[origin["Title"].isin(titles)]

  result["Similarity"] = [molist[i][1] for i in range(len(molist))]

  return result

output(title, limit)

,MovieID,Title,Genre,Similarity
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy,1.0
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy,1.0
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy,1.0
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy,1.0
1556,2093,Return to Oz (1985),Adventure Children Fantasy,1.0
1617,2161,"NeverEnding Story, The (1984)",Adventure Children Fantasy,1.0
1618,2162,"NeverEnding Story II: The Next Chapter, The (1...",Adventure Children Fantasy,1.0
1799,2399,Santa Claus: The Movie (1985),Adventure Children Fantasy,1.0
3574,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure Children Fantasy,1.0


In [29]:
title = origin.iloc[3574:3575]

title

,MovieID,Title,Genre
3574,4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure Children Fantasy


In [30]:
title = origin.iloc[3574, 1]

output(title, 5)

,MovieID,Title,Genre,Similarity
1,2,Jumanji (1995),Adventure Children Fantasy,1.0
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy,1.0
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy,1.0
767,1009,Escape to Witch Mountain (1975),Adventure Children Fantasy,1.0
1514,2043,Darby O'Gill and the Little People (1959),Adventure Children Fantasy,1.0


`Cold Start Problem`

Let's Try Tackle Cold Start Problem

In [31]:
option = ["Children", "Fantasy"]

option

['Children', 'Fantasy']

In [32]:
def starter(option, limit):

  result = []

  ontari = origin["Genre"].unique()

  for item in ontari:

    spliter = item.split(" ")

    overlap = len(set(option) & set(spliter)) / len(spliter)

    if overlap > 0.5:

      result.append((item, overlap))

  result = sorted(result, key=lambda i:i[1], reverse=reverse)[:limit]

  result = [i[0] for i in result]

  result = origin[origin["Genre"].isin(result)].iloc[:limit]

  return result

starter(option, 5)

,MovieID,Title,Genre
1,2,Jumanji (1995),Adventure Children Fantasy
53,60,"Indian in the Cupboard, The (1995)",Adventure Children Fantasy
109,126,"NeverEnding Story III, The (1994)",Adventure Children Fantasy
209,243,Gordy (1995),Children Comedy Fantasy
301,343,"Baby-Sitters Club, The (1995)",Children


Our User, Choose Gordy

In [33]:
title = origin.iloc[209, 1]

output(title, 5)

,MovieID,Title,Genre,Similarity
313,355,"Flintstones, The (1994)",Children Comedy Fantasy,1.0
355,410,Addams Family Values (1993),Children Comedy Fantasy,1.0
637,810,Kazaam (1996),Children Comedy Fantasy,1.0
649,837,Matilda (1996),Children Comedy Fantasy,1.0
